<center>
<a href="http://www.insa-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo-insa.jpg" style="float:left; max-width: 120px; display: inline" alt="INSA"/></a> 

<a href="http://wikistat.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/wikistat.jpg" style="max-width: 250px; display: inline"  alt="Wikistat"/></a>

<a href="http://www.math.univ-toulouse.fr/" ><img src="http://www.math.univ-toulouse.fr/~besse/Wikistat/Images/logo_imt.jpg" style="float:right; max-width: 250px; display: inline" alt="IMT"/> </a>
</center>

# Text Mining et Catégorisation de Produits avec Python et Scikit-learn

## Introduction

Il s'agit d'une version simplifiée du concours proposé par CDiscount et paru sur le site [datascience.net](https://www.datascience.net/fr/challenge). Les données d'apprentissage sont accessibles sur demande auprès de CDiscount. Le solutions de l'échantillon test du concours ne sont pas et ne seront pas rendues publiques. Un échantillon test est donc construit pour l'usage de ce tutoriel.  L'objectif est de prévoir la catégorie d'un produit à partir de son descriptif. Seule la catégorie principale (1er niveau) est prédite au lieu des trois niveaux demandés dans le concours. L'objectif est plutôt de comparer les performances des méthodes et technologies en fonction de la taille de la base d'apprentissage ainsi que d'illustrer sur un exemple complexe le prétraitement de données textuelles. La stratégie de sous ou sur échantillonnage des catégories qui permet d'améliorer la prévision n'a pas été mise en oeuvre.
* L'exemple est présenté sur un échantillon réduit d'un million de produits au lieu des 15M initiaux
* L'échantillon réduit peut encore l'être puis séparé en 2 parties: apprentissage et validation. 
* Les données textuelles sont  nettoyées, racinisées, vectorisées avant modélisation.
* Trois modélisations sont estimées: logistique, arbre, forêt aléatoire.
* Optimiser l'erreur en faisant varier différents paramètres: types et paramètres de vectorisation (TF-IDF), paramètres de la régression logistique (pénalisation L1) et de la forêt aléatoire (nombre d'arbres et nombre de variables aléatoire).

Exécuter finalement le code pour différentes tailles (paramètre tauxTot ci-dessous) de l'échantillon d'apprentissage et comparer les qualités de prévision obtenues. 

Deux échantillons de test ont été mis de côté et seront utilisés dans un prochain calepin (avec pyspark) pour comparer les stratégies.


## Fonctions nécessaires à la préparation des données

In [1]:
#Importation des librairies utilisées
import unicodedata 
import time
import pandas as pd
import numpy as np
import random
import nltk
import collections
import csv

In [2]:
# Définition des noms de fichier
## Définition du répertoire de travail
## par défaut le répertoire courant
DATA_DIR = ""
#Fichier réduit avec un million d'articles
training_reduit_path = "CDiscount_reduit.csv"
## Fichiers résultant de l'extraction apprentissage/validation
training_reduit_train_path = DATA_DIR + "training_reduit_train.csv"
training_reduit_validation_path = DATA_DIR + "training_reduit_validation.csv"
## Fichiers nettoyés 
training_clean_train_path = DATA_DIR + "training_clean_train.csv"
training_clean_validation_path = DATA_DIR + "training_clean_validation.csv"

In [3]:
# Liste des variables dans les différents fichiers d'apprentissage et de test
HEADER_TEST = ['Description','Libelle','Marque']
HEADER_TRAIN =['Categorie1','Categorie2','Categorie3','Description','Libelle','Marque']

In [4]:
# Si nécessaire (première exécution) chargement de nltk, librairie pour la suppression 
## des mots d'arrêt et la racinisation
## nltk.download()

In [5]:
# Fonction pour extraire aléatoirement tauxTot lignes d'un fichier et 
## réaliser le partage aléatoire en deux sous-fichiers validation (tauxValid) 
## et apprentissage (1-tauxValid)
def split_dataset(input_path,output_train_path,output_validation_path,tauxTot,tauxValid):
    print("Split Start")
    random.seed(11)
    time_start = time.time()
    with open(input_path) as fInput:
        with open(output_train_path, "w") as fTrain, open(output_validation_path, "w") as fValid:
            for line in fInput:
                if random.random() < tauxTot:
                    fTrain.write(line) if random.random() > tauxValid else fValid.write(line)      
    time_end = time.time()
    print("Split Takes %d s" %(time_end-time_start))
    return

In [6]:
# Fonction clean : nettoyage préalable a l'apprentissage statistique des fichiers 
from bs4 import BeautifulSoup
import re
import nltk
## listes de mots à supprimer dans la description des produits
nltk_stopwords = nltk.corpus.stopwords.words('french')
lucene_stopwords = [unicode(w, "utf-8") for w in open(DATA_DIR+"lucene_stopwords.txt").read().split(",")]
stopwords = list(nltk_stopwords)
## Algo de stemming permettant de supprimer automatiquement les 
## suffixes pour n'obtenir que la racine des mots.
## On parle de racinisation.
stemmer=nltk.stem.SnowballStemmer('french')

In [7]:
# fonction de nettoyage des mots, 
## tout en minuscule, caractères spéciaux, numériques
## racinisation
def clean_txt(txt):
    ### remove html stuff
    txt = BeautifulSoup(txt,from_encoding='utf-8').get_text()
    ### lower case
    txt = txt.lower()
    ### special escaping character '...'
    txt = txt.replace(u'\u2026','.')
    txt = txt.replace(u'\u00a0',' ')
    ### remove accent btw
    txt = unicodedata.normalize('NFD', txt).encode('ascii', 'ignore')
    ###txt = unidecode(txt)
    ### remove non alphanumeric char
    txt = re.sub('[^a-z_]', ' ', txt)
    ### remove french stop words
    tokens = [w for w in txt.split() if (len(w)>2) and (w not in nltk_stopwords)]
    ### french stemming
    tokens = [stemmer.stem(token) for token in tokens]
    ### tokens = stemmer.stemWords(tokens)
    return ' '.join(tokens)

In [8]:
# fonction de nettoyage du fichier(stemming et liste de mots à supprimer)
def clean_file(input_path, output_path, type, columns= 
               ('Description', 'Libelle', 'Marque') , nrows = None):
    print("Clean File: " + input_path)
    if type=="train":
        header = HEADER_TRAIN
        idx_id = 3
    elif type == "test":
        header = HEADER_TEST
        idx_id = 0
    else:
        raise ValueError("Type should be either 'test' or 'train', not " + type)
    columns_idx  = [(k, v) for v,k in enumerate(header) if k in columns]
    ff = open(output_path,'w')
    line = ';'.join(header[:idx_id] + list(columns))
    ff.write(line+'\n')
    start_time = time.time()
    counter = 0
    for line in open(input_path):
        if counter==0:
            counter+=1
            continue
        ls = line.split(';')
        ls_out = ls[:idx_id]
        for k,v in columns_idx:
            if k =="Marque":
                txt = ls[v]
                txt = re.sub('[^a-zA-Z0-9]', '_', txt).lower()
                ls_out.append(txt)
            else:
                txt = ls[v]
                ls_out.append(clean_txt(txt))
        line = ';'.join(ls_out)
        ff.write(line+'\n')
        counter += 1
        if (nrows is not None) and (counter>=nrows):
            break
    ff.close()
    return

In [9]:
#Fonction vectorizer
## Création d’une matrice indiquant
## les fréquences" des mots contenus dans chaque description
## de nombreux paramètres seraient à tester
from sklearn.feature_extraction.text import TfidfVectorizer
def vectorizer(df,columns,stop_words=None):
    txt=np.repeat("",len(df)).astype("object")
    print(txt.dtype)
    for c in columns:
        txt+=" " + df[c].values
    vec = TfidfVectorizer(
        min_df = 1,
        stop_words = stop_words,
        smooth_idf=True,
        norm='l2',
        sublinear_tf=True,
        use_idf=True,
        ngram_range=(1,2)) #bi-grams
    X = vec.fit_transform(txt)
    return vec,X

## Exécution de la préparation des données

### Extraction des fichiers, nettoyage

In [12]:
#Fichier "main", on procede au mélange, au partitionnage, 
## à l'échantillonnage, au nettoyage, à la vectorisation
# Définition des taux d'échantillonnage pour fixer la taille des fichiers
tauxTot=0.01  # part totale extraite du fichier initial ici déjà réduit
tauxValid=0.20 # part de l'échantillon de validation
# liste des variables prises en compte
columns = ('Description', 'Libelle') # laisse tomber la marque
# Extraction d'un sous-échantillon séparé en deux parties 
## apprentissage et validation
split_dataset(training_reduit_path,training_reduit_train_path,
                  training_reduit_validation_path, tauxTot, tauxValid)
# nettoyage des fichiers
clean_file(training_reduit_train_path, training_clean_train_path,
               "train", columns= columns)
clean_file(training_reduit_validation_path, training_clean_validation_path,
               "train", columns= columns)

Split Start
Split Takes 0 s
Clean File: training_reduit_train.csv
Clean File: training_reduit_validation.csv


### Vectorisation

In [13]:
# Vectorisation des données textuelles
## Attention d'autres options de la fonction vectorizer seraient à tester à ce niveau
## lire la base d'apprentissage
dftrain = pd.read_csv(training_clean_train_path, sep = ";").fillna("")
## vectorisation de l'apprentissage
vec,X = vectorizer(dftrain, columns, None)
Y = dftrain['Categorie1'].values
## lire les données de validation
dfvalid=pd.read_csv(training_clean_validation_path,sep=";").fillna("")
txt=np.repeat("",len(dfvalid)).astype("object")
for c in columns:
    txt+=" " + dfvalid[c].values
## application de la vectorisation 
Xv=vec.transform(txt)  
Yv=dfvalid['Categorie1'].values

object


## Modélisation et performances

In [14]:
# Regression Logistique 
## estimation
from sklearn.linear_model import LogisticRegression
cla = LogisticRegression(C=100)
cla.fit(X,Y)
score=cla.score(X,Y)
print('# training score:',score)

('# training score:', 0.99962316291923126)


In [15]:
## erreur en validation
scoreValidation=cla.score(Xv,Yv)
print('# validation score:',scoreValidation)

('# validation score:', 0.86011150532184488)


In [16]:
#Méthode  CART
from sklearn import tree
clf = tree.DecisionTreeClassifier()
time_start = time.time()
clf = clf.fit(X, Y)
time_end = time.time()
print("CART Takes %d s" %(time_end-time_start) )
score=clf.score(X,Y)
print('# training score :',score)

CART Takes 7 s
('# training score :', 0.99962316291923126)


In [17]:
scoreValidation=clf.score(Xv,Yv)
print('# validation score :',scoreValidation)

('# validation score :', 0.69285352255448551)


In [18]:
# Random forest
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100,n_jobs=-1,max_features=24)
Y = dftrain['Categorie1'].values
time_start = time.time()
rf = rf.fit(X, Y)
time_end = time.time()
print("RF Takes %d s" %(time_end-time_start) )
score=rf.score(X,Y)
print('# training score :',score)

RF Takes 15 s
('# training score :', 0.99962316291923126)


In [19]:
scoreValidation=rf.score(Xv,Yv)
print('# validation score :',scoreValidation)

('# validation score :', 0.77749619868220987)
